In [10]:
import numpy as np
import json
import pandas as pd
import matplotlib as plt
import seaborn as sns
import csv
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler, Stream, API, Cursor
import requests

In [11]:
## access token informations 
access_token1 = "1222829174802538496-AXkAccE1dWMjuEFlEYChpgOibc6SbF"
access_token_secret1 = "jtr66CvVvspsANplSBpuMEQR5iLwK2fRtzM6aDhaC1rZT"

consumer_key1 = "sL9E5QL83DVEshfttLjEEj930"
consumer_secret1 = "R9L4Ar7UWTKB4WQw7cEoybumyTInZ6pYP3cQ5GFlWYFllYAeec"

In [12]:
auth = OAuthHandler(consumer_key1, consumer_secret1)
auth.set_access_token(access_token1, access_token_secret1)
api = API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [13]:
class MyStreamListener(StreamListener):
    """
    Twitter listener, collects streaming tweets and output to a file
    """
    def __init__(self, api, OUTPUT_FILENAME, stop_condition=10):
        """
        initialize the stream, with num. of tweets and saving the outputfile
        """
        
        # this line is needed to import the characteristics of the streaming API
        super(MyStreamListener, self).__init__()
        
        # to-count the number of tweets collected        
        self.num_tweets = 0
        
        # save filename
        self.filename = OUTPUT_FILENAME
        
        # stop-condition
        self.stop_condition = stop_condition
        

    def on_status(self, status):
        
        """
        this function runs each time a new bunch of tweets is retrived from the streaming
        """
        
        with open(self.filename, "a+") as f:
            tweet = status._json
            
            f.write(json.dumps(tweet) + '\n')
            #self.output.append(tweet)
            self.num_tweets += 1
        
            # Stop condition        
            if self.num_tweets <= self.stop_condition:
                return True
            else:
                return False
        

    def on_error(self, status):
        """
        function useful to handle errors. It's possible to personalize it 
        depending on the way we want to handle errors
        """
        
        print(status)
        #returning False in on_error disconnects the stream
        return False

In [14]:
OUTPUT_FILENAME = "other-outputs/tweets_US_Election_2020.json"
stop_condition = 500

l = MyStreamListener(api, OUTPUT_FILENAME, stop_condition)
# here we recall the Stream Class from Tweepy to input the authentication info and our personalized listener 
stream = Stream(auth=api.auth, listener=l)


# keywords we may want decide to track 
TRACKING_KEYWORDS = ["Donald", "Trump", "Joe", "Biden", "America", "USA"]
stream.filter(
    track=TRACKING_KEYWORDS, 
    is_async=False, 
    languages = ["en"]
)

In [15]:
json_Election_file = []
for line in open(OUTPUT_FILENAME, 'r'):
    json_Election_file.append(json.loads(line))

In [17]:
dict_train={}
i=0
for train_file in json_Election_file:
    dict_train[i]=train_file
    i+=1
train = pd.DataFrame.from_dict(dict_train, orient='index')
train.reset_index(level=0, inplace=True)

In [31]:
pd.set_option('display.max_columns', None)
train["text"].head()

0    RT @iamjumpingin: 1st they choose Biden to be ...
1    RT @LLinWood: The Georgia Governor &amp; Secre...
2    RT @Forecast432hz: NO SHIT 👇🏼\nFacebook, Twitt...
3    @violethaze10 @rhonda0228 @NewDayForNJ Yeah I ...
4    RT @MoralScold: @RealMattCouch God, please giv...
Name: text, dtype: object

In [ ]:
#Normalize functions:
def stemming(text: str) -> None:
    pass
def punctuation_removal(text: str) -> None:
    pass
def remove_stop_words(text: str) -> None:
    pass